In [1]:
import pandas as pd
import numpy as np
import os
import shutil
from subprocess import call, run
import json

import matplotlib.pyplot as plt
import seaborn as sns

from scipy.optimize import curve_fit

import sys
sys.path.append('/home/sirius/00_ikt/данные для начала работы/swapDirections_example')
import DRA_utils_v2 as dra

def parse_params(fpath):
    """
    fpath : str (path to the file to parse)
    ------
    returns: {'total_porosity': value, 'absolute_permeability' : value, 'formation_factor': value} 
    
    """
    lines = []
    with open(fpath) as f:
        lines = f.readlines()
    for line in lines:
        if 'Total porosity' in line :
            total_porosity = float(line.split()[-1])
        if 'Absolute permeability' in line:
            absolute_permeability = float(line.split()[-2])
        if 'Formation factor' in line:
            formation_factor = float(line.split()[-1])
            
    return {'total_porosity': total_porosity, 'absolute_permeability' : absolute_permeability, 'formation_factor': formation_factor}

### Блок обмена осей

In [3]:
path = '/home/sirius/00_ikt/data_parsing/carbonate'

for root, dirs, files in os.walk(path):
    for direc in dirs:
        if direc != 'data':
            new_path = os.path.join(root, direc)

            for _, _, files in os.walk(new_path):
                for file in files:
#                     Проходимся по всем файлам с расширением .mhd и меняем ось, сохраняя новые файлы
                    if (file.endswith('.mhd')) and ('_VElems' not in file):
                        print(file)
                        filename = file.replace('.mhd', '')
                        dra.swapDirections(new_path + f'/{file}', new_path + f'/{filename}_swapXY.mhd', 'XY')
                        dra.swapDirections(new_path + f'/{file}', new_path + f'/{filename}_swapXZ.mhd', 'XZ')

Ketton_1000c_3p00006um.mhd
15.mhd
10.mhd
25.mhd
9.mhd
0.mhd
5.mhd
12.mhd
21.mhd
26.mhd
2.mhd
16.mhd
22.mhd
7.mhd
24.mhd
6.mhd
23.mhd
14.mhd
17.mhd
4.mhd
20.mhd
8.mhd
11.mhd
18.mhd
13.mhd
19.mhd
1.mhd
3.mhd


### Блок симуляции через солвер

In [2]:
path = '/home/sirius/00_ikt/data_parsing/carbonate'

for root, dirs, files in os.walk(path):
    for direc in dirs:
        if direc != 'data':
            new_path = os.path.join(root, direc)

            for _, _, files in os.walk(new_path):
                for file in files:
#                         В папке с кернами по всем осям необходимо добавить 2 файла:
#                                         - input_pnflow_original.dat
#                                         - input_pnflow.dat - сюда для каждого керна записываются описательные параметры
                    if ('.mhd' in file) and ('_VElems' not in file):
                        print(file)
                        filename = file.replace('.mhd', '')

                        with open(new_path + '/input_pnflow_original.dat', 'r') as inp:
                            llist = []
                            for line in inp:
                                if line.startswith('NETWORK'):
                                    line = f'NETWORK  F {filename};   // the base name for of the network file, without _link1.dat, _link2, _pore1'
                                if line.startswith('TITLE'):
                                    line = f'TITLE  {filename};  // base name for the output files'
                                llist.append(line)

                        with open(new_path + '/input_pnflow.dat', 'w') as inp:
                            inp.writelines(llist)
#                             Запуск симуляции
                        pnextract = os.system(f'cd {new_path}; /home/sirius/00_ikt/software/pnflow-master_commit20210823/bin/pnextract {file} > log.pnextract; /home/sirius/00_ikt/software/pnflow-master_commit20210823/bin/pnflow input_pnflow.dat > log.pnflow')    

Ketton_1000c_3p00006um.mhd


Killed


Ketton_1000c_3p00006um_swapXZ.mhd


Killed


Ketton_1000c_3p00006um_swapXY.mhd


...

15.mhd
10.mhd
25.mhd
9.mhd
0.mhd
5.mhd
8_swapXZ.mhd
12.mhd
23_swapXY.mhd
21.mhd
26.mhd
11_swapXZ.mhd
2.mhd
24_swapXY.mhd
16.mhd
8_swapXY.mhd
12_swapXZ.mhd
0_swapXZ.mhd
22.mhd
20_swapXZ.mhd
22_swapXZ.mhd
7.mhd
16_swapXZ.mhd
12_swapXY.mhd
24.mhd
19_swapXY.mhd
18_swapXZ.mhd
6.mhd
1_swapXY.mhd
7_swapXY.mhd
6_swapXZ.mhd
17_swapXY.mhd
22_swapXY.mhd
20_swapXY.mhd
23.mhd
9_swapXZ.mhd
23_swapXZ.mhd
24_swapXZ.mhd
17_swapXZ.mhd
9_swapXY.mhd
18_swapXY.mhd
15_swapXY.mhd
10_swapXZ.mhd
5_swapXY.mhd
26_swapXY.mhd
13_swapXY.mhd
7_swapXZ.mhd
16_swapXY.mhd
26_swapXZ.mhd
2_swapXZ.mhd
14.mhd
25_swapXZ.mhd
6_swapXY.mhd
21_swapXY.mhd
17.mhd
25_swapXY.mhd
0_swapXY.mhd
14_swapXY.mhd
11_swapXY.mhd



  Warning in addKcSwKrsQs, FlowData.h:162:  KrQsss_[icy].back()[3]<=KcSwKrsQs[3]+1e-8


10_swapXY.mhd
4.mhd
20.mhd
5_swapXZ.mhd
8.mhd
11.mhd
18.mhd
14_swapXZ.mhd
13.mhd
19.mhd
19_swapXZ.mhd
15_swapXZ.mhd
2_swapXY.mhd
13_swapXZ.mhd
1.mhd
3_swapXZ.mhd
3.mhd
Estaillades_1000c_3p31136um.mhd_
21_swapXZ.mhd




 Error in pnextract: 
   

  Error in readFromHeader, /home/avdonin/pnflow-master_commit20210823/src/libvoxel/voxelImage.cpp:749:  
   Unknown (header) file type: Estaillades_1000c_3p31136um.mhd_

 Aborting! 



4_swapXY.mhd
4_swapXZ.mhd
1_swapXZ.mhd
3_swapXY.mhd


### Блок записи параметров симуляции (absolutePermeability, totalPorosity, 

In [4]:
path = '/home/sirius/00_ikt/data_parsing/carbonate'
params_dict = {}
for root, dirs, files in os.walk(path):
    for direc in dirs:
        if direc != 'data':
            new_path = os.path.join(root, direc)
            for _, _, files in os.walk(new_path):
                for file in files:
                    if (file.endswith('pnflow.prt')) and ('_VElems' not in file):
                        try:
                            filename = file.replace('_pnflow.prt', '')
                            prt_path = os.path.join(new_path, file)
                            params = parse_params(prt_path)
                            params_dict[prt_path] = params
                        except UnboundLocalError:
                            continue
                            
# Запись параметров в папку /parameters
with open(f'{path}/parameters/parameters_carbonates.json', "w") as file:
    json.dump(params_dict, file)

### Блок парсинга параметров ОФП: а, b, krw_max, soi

In [11]:
path = '/home/sirius/00_ikt/data_parsing/carbonate'
sns.set_theme()
myEps=1e-6 # define small number for kro cut-off

a_list = []
b_list = []
krw_list = []
kro_list = []
swc_list = []
sor_list = []
path_list = []

def krw_func(Se,a):
    return krwMax*Se**a

def kro_func(Se,b):
    return kroMax*(1-Se)**b

for root, dirs, files in os.walk(path):
        for direc in dirs:
            if direc != 'data':
                new_path = os.path.join(root, direc)
                for _, _, files in os.walk(new_path):
                    for file in files:
                        if file.endswith('cycle2_imb.out'):
                            filepath = new_path + f"/{file}"
                            df = pd.read_csv(filepath, skiprows=3, sep='\t', header=None)
                            df.rename(columns={0: "Sw",1: "Pc(Pa)",2:"Krw",3:"Kro",4:"RI",5:"CC"},inplace=True)
                            # cut off all points, where kro<myEps
                            cutOffIndex=df.loc[df['Kro']<=1e-6].first_valid_index()
                            if cutOffIndex != 0:
                                path_list.append(filepath)
                                df = df.iloc[0:cutOffIndex]
                                Sor = 1 - df['Sw'].iloc[-1]
                                sor_list.append(Sor)
                                Swc = df["Sw"][0]
                                swc_list.append(Swc)
                                krwMax = np.max(df["Krw"])
                                krw_list.append(krwMax)
                                kroMax = np.max(df["Kro"])
                                kro_list.append(kroMax)
                                df['Se'] = (df['Sw'] - Swc)/(1 - Sor - Swc)

                                a, a_pcov = curve_fit(krw_func, df['Se'].values, df['Krw'].values,p0=1,maxfev=10000)
                                b, b_pcov = curve_fit(kro_func, df['Se'].values, df['Kro'].values,p0=1,maxfev=10000)

                                a_list.append(a)
                                b_list.append(b)
                            else:
                                continue

/tmp/ipykernel_1546339/3802550575.py:14: RuntimeWarning: divide by zero encountered in power
  return krwMax*Se**a
/tmp/ipykernel_1546339/3802550575.py:17: RuntimeWarning: invalid value encountered in power
  return kroMax*(1-Se)**b
/home/sirius/.local/lib/python3.8/site-packages/scipy/optimize/minpack.py:807: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/tmp/ipykernel_1546339/3802550575.py:17: RuntimeWarning: divide by zero encountered in power
  return kroMax*(1-Se)**b


### Блок записи параметров ОФП

In [15]:
params = pd.DataFrame()
params['path'] = path_list
params['a'] = list(map(lambda x: x[0], a_list))
params['b'] = list(map(lambda x: x[0], b_list))
params['krw_max'] = krw_list
params['kro_max'] = kro_list
params['swc'] = swc_list
params['sor'] = sor_list

params.to_csv('/home/sirius/00_ikt/data_parsing/carbonate/parameters/ab_coeffs_carbonates.csv')